In [355]:
import numpy as np
from numba.experimental import jitclass
from numba import float32
import matplotlib.pyplot as plt

In [408]:
ACO_spec = [
    ("population", float32[:, :]),
    ("dist", float32[:]),
    ("pheromon", float32[:, :]),
    ("adjacency_matrix", float32[:, :])
]


class ACO:
    def __init__(self, num_pop, graph):
        n = len(graph)

        self.population = [[0]] * num_pop # Each ant starts at city number 0
        self.dist = np.zeros(shape=(num_pop)) # dist[ant] = distance made so far

        self.pheromon = np.random.rand(n, n) # Some initialization, really

        self.adjacency_matrix = np.zeros(shape=(n, n), dtype=np.float32) # [i, j] = Euclidean distance between city i and j

        # Setting up the distances
        for i in range(n):
            self.adjacency_matrix[i] = np.apply_along_axis(np.linalg.norm, 1, graph[i] - graph)

        # Banning paths from city to itself 
        self.adjacency_matrix = np.where(self.adjacency_matrix == 0, np.inf, self.adjacency_matrix)

    def main_loop(self, num_iter, alpha, beta, Q, rho):
        n = self.adjacency_matrix.shape[0]
        best_route = None
        min_dist = np.inf
        
        mu = 1 / self.adjacency_matrix

        for i in range(num_iter):
            self.population = [[0]] * len(self.population) # Each ant starts at city number 0
            self.dist = np.zeros(self.dist.shape)
            delta_tau = np.zeros_like(self.pheromon) # To perform pheromon decay after loop ends
            
            for ant in range(len(self.population)):
                # Looping until ant hasn't visited all of the cities
                while len(self.population[ant]) != n:
                    # Getting the last city that ant have visited so far
                    current_city = self.population[ant][-1]

                    # Temporary variable to compute probabilities later
                    tmp = (self.pheromon[current_city] ** alpha) * (mu[current_city] ** beta)

                    # Setting visited cities probabilities to zero
                    for j in range(n):
                        if j in self.population[ant]:
                            tmp[j] = 0

                    # Probabilities itself
                    prob = tmp / tmp.sum() # nan's since it counts current city for wich mu = inf

                    # Cumulative probabilites to choose next city
                    cum_prob = np.cumsum(prob)

                    # Choosing next city while it hasn't been visited
                    next_city = np.searchsorted(cum_prob, np.random.rand())

                    while next_city in self.population[ant]:
                        next_city = np.searchsorted(cum_prob, np.random.rand())

                    self.population[ant].append(next_city)
                    self.dist[ant] += self.adjacency_matrix[current_city][next_city]

                    # Updating pheromon
                    self.pheromon[current_city][next_city] += Q / self.dist[ant]
                    delta_tau[current_city][next_city] += Q / self.dist[ant]

                    if len(self.population[ant]) == n:
                        # Connecting the last city from the first
                        self.dist[ant] += self.adjacency_matrix[0][self.population[ant][-1]]

                # There's some bug which I don't know how fix
                if min_dist > self.dist[ant] and self.dist[ant] != 0:
                    min_dist = np.copy(self.dist[ant])
                    best_route = np.copy(self.population[ant])

            print("Iteration: %i\nMin distance: %.3f\nMin route %s" % (i + 1, min_dist, best_route)) 

            # Pheromone decay
            self.pheromon = (1 - rho) * self.pheromon + delta_tau
        
        return min_dist, best_route

In [402]:
def create_circle(n, radius=1):
    circle = []
    theta = 0
    delta = 2 * np.pi / n

    side_length = 2 * radius * np.sin(delta / 2)

    for _ in range(n):
        x = np.cos(theta)
        y = np.sin(theta)
        circle.append(np.array([x, y]))
        theta += delta


    return np.array(circle), n * side_length

In [429]:
file_path = r'D:\Opera_Download\xqf131.tsp'
def read_tsp_file(filepath):
    tsp = []
    with open(filepath) as file:
        for _ in range(8):
            file.readline()
        
        for line in file.readlines()[:-1]:
            _, x, y = line.split()
            tsp.append(np.array([np.float32(x), np.float32(y)]))

    return np.array(tsp)

In [432]:
"""
Min distance: 6.868
Min route [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 17 19]
Real ans 6.257
"""
tsp1 = read_tsp_file(file_path)
test = ACO(500, tsp1)
test.main_loop(num_iter=10_000, alpha=1, beta=0.5, rho=0.01, Q=564)

Iteration: 1
Min distance: 3584.966
Min route [  0  57  35  91  19  79  78  21  58   7  76  99 117  60  73  48  14  16
  17  74 128 108  41  90  97  15  34  10  37  86  96 110 115 118  65 121
   8  38  70 104  32   2  29  12  77  92  88  75  83  40  62  68  61  54
  22 111 129 116  36  30  59  31  27 123  49 109  72  51  80  18   6   3
  64  50 105  13   5  25  52   4 114 119 106 103 101 125 112   9 126  69
  33  55  44  45  63  84  82  98  26  46  47 113 120  81  87  66  24  11
 122 127  71  43  39  23  42  95 130  85  56  94   1  28  20  67  53 100
 124  93 107 102  89]
Iteration: 2
Min distance: 3584.966
Min route [  0  57  35  91  19  79  78  21  58   7  76  99 117  60  73  48  14  16
  17  74 128 108  41  90  97  15  34  10  37  86  96 110 115 118  65 121
   8  38  70 104  32   2  29  12  77  92  88  75  83  40  62  68  61  54
  22 111 129 116  36  30  59  31  27 123  49 109  72  51  80  18   6   3
  64  50 105  13   5  25  52   4 114 119 106 103 101 125 112   9 126  69
  33  55  

In [230]:
arr = np.array([0.1, 0, 0, 0.4, 0, 0, 0.5])
np.cumsum(arr)

array([0.1, 0.1, 0.1, 0.5, 0.5, 0.5, 1. ])

In [175]:
((1 - 0.30901699) ** 2 + 0.95105625 ** 2) ** .5

1.1755702917191821